# Semantic Kernel에서 여러 LLM 사용해보기

Semantic Kernel은 OpenAI뿐만 아니라 **Hugging Face**, **Llama**, **Gemini**와 같은 다양한 대형 언어 모델(LLM)을 지원하여 개발자들이 여러 AI 모델을 손쉽게 통합하고 활용할 수 있도록 돕습니다. 심지어 LM Studio, Ollama 및 직접 Local LLM을 연결시키는 등의 Local LLM도 연결 할 수 있습니다. 그리고 Kernel에 다른 코드를 그대로 유지 하면서 모델만 바꿔서 사용할 수도 있습니다.



In [1]:
#!import config/Settings.cs
var settings = Settings.LoadFromFile();

#r "nuget: Microsoft.SemanticKernel"

Installed Packages Microsoft.SemanticKernel, 1.26.0

Azure OpenAI의 자격 증명을 기반으로 Kernel을 생성합니다.

In [2]:
// Import namespaces
using Microsoft.SemanticKernel;

var kernel = Kernel.CreateBuilder()
                   .AddAzureOpenAIChatCompletion(
                       endpoint: settings.AzureOpenAIEndpoint,
                       apiKey: settings.AzureOpenAIApiKey,
                       deploymentName:  "gpt-4o")
                   .Build();

In [3]:
var prompt = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Prompt");

var result = await kernel.InvokePromptAsync(prompt);

Console.WriteLine(result);

저는 OpenAI가 개발한 언어 모델인 ChatGPT입니다. 질문에 답변하고 대화를 나누는 데 도움을 주기 위해 설계되었어요. 어떤 도움이 필요하신가요?


## Google Gemini 모델 사용

Google의 Gemini 모델을 사용해보겠습니다. `Microsoft.SemanticKernel.Connectors.Google` 누겟 패키지 설치가 필요하며 Kernel을 빌드할 때 `AddGoogleAIGeminiChatCompletion` 메서드에 모델과 키값만 넣어주면 됩니다.

In [4]:
#r "nuget: Microsoft.SemanticKernel.Connectors.Google, 1.26.0-alpha"

Installed Packages Microsoft.SemanticKernel.Connectors.Google, 1.26.0-alpha

In [5]:
#pragma warning disable SKEXP0070

var kernel = Kernel.CreateBuilder()
            .AddGoogleAIGeminiChatCompletion("gemini-1.5-flash", settings.GoogleGeminiKey)
            .Build();

In [6]:
var prompt = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Prompt");

var result = await kernel.InvokePromptAsync(prompt);

Console.WriteLine(result);

저는 구글에서 개발한 대규모 언어 모델입니다.


## Hugging Face 모델 사용

Semantic Kernel로 Hugging Face의 다양한 모델을 사용할 수 있습니다. `Microsoft.SemanticKernel.Connectors.HuggingFace` 누겟 패키지 설치가 필요하며 Kernel을 빌드할 때 `AddHuggingFaceChatCompletion` 메서드에 모델과 키값을 넣어주면 됩니다. Hugging Face 모델의 활용 방법은 다양하기 때문에 다양한 방식으로 활용이 가능하고, 때로는 모델을 사용하기 위해 여러 작업들이 필요할 수도 있습니다.

여기에서는 [https://huggingface.co/microsoft/Phi-3.5-mini-instruct](https://huggingface.co/microsoft/Phi-3.5-mini-instruct) 모델을 사용했습니다.

In [7]:
#r "nuget: Microsoft.SemanticKernel.Connectors.HuggingFace, 1.26.0-preview"

Installed Packages Microsoft.SemanticKernel.Connectors.HuggingFace, 1.26.0-preview

In [8]:
#pragma warning disable SKEXP0070

var kernel = Kernel.CreateBuilder()
                   .AddHuggingFaceChatCompletion("microsoft/Phi-3.5-mini-instruct", apiKey: settings.HuggingFacePhiApiKey)
                   .Build();

In [9]:
var prompt = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Prompt");

var result = await kernel.InvokePromptAsync(prompt);

Console.WriteLine(result);

I am Phi, created by Microsoft. I'm an AI digital assistant designed to provide information, answer questions, and engage in conversation to help users with their queries.


# 여러 인스턴스를 생성해 동시에 사용

Semantic Kernel에서 여러 인스턴스를 생성하여 사용하는 방법을 예제로 설명하겠습니다. 이는 다양한 모델 혹은 설정, 기능을 가진 커널을 동시에 활용하고자 할 때 유용합니다.

In [10]:
#pragma warning disable SKEXP0070
#pragma warning disable SKEXP0001

// 첫 번째 커널 인스턴스 생성
var kernel1 = Kernel.CreateBuilder()
            .AddAzureOpenAIChatCompletion( 
                       endpoint: settings.AzureOpenAIEndpoint,
                       apiKey: settings.AzureOpenAIApiKey,
                       deploymentName:  "gpt-4o")
            .Build();

// 두 번째 커널 인스턴스 생성
var kernel2 = Kernel.CreateBuilder()
            .AddGoogleAIGeminiChatCompletion("gemini-1.5-flash", settings.GoogleGeminiKey, serviceId: "gemini")
            .Build();

var prompt = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Prompt");

// 첫 번째 커널을 사용하여 작업 수행
var result1 = await kernel1.InvokePromptAsync(prompt);
Console.WriteLine($"첫 번째 커널 결과: {result1}");

// 두 번째 커널을 사용하여 작업 수행
var result2 = await kernel2.InvokePromptAsync(prompt);
Console.WriteLine($"두 번째 커널 결과: {result2}");

첫 번째 커널 결과: 안녕하세요! 저는 OpenAI에서 개발한 AI 언어 모델인 GPT-3입니다. 텍스트 기반 질의응답, 대화, 정보 제공 등을 도와드릴 수 있습니다. 어떤 도움이 필요하신가요?
두 번째 커널 결과: 저는 구글에서 훈련된 대규모 언어 모델입니다. 



## 하나의 커널 인스턴스에서 여러 모델을 동시에 사용

모델을 등록할 때 serviceId를 지정할 수 있고, 특정 serviceId를 지정해서 호출할 수 있습니다.

In [11]:
#pragma warning disable SKEXP0070
#pragma warning disable SKEXP0001

var kernel = Kernel.CreateBuilder()
                   .AddAzureOpenAIChatCompletion( 
                       endpoint: settings.AzureOpenAIEndpoint,
                       apiKey: settings.AzureOpenAIApiKey,
                       deploymentName:  "gpt-4o", serviceId: "gpt")
                    .AddGoogleAIGeminiChatCompletion("gemini-1.5-flash", settings.GoogleGeminiKey, serviceId: "gemini")
                    .AddHuggingFaceChatCompletion("microsoft/Phi-3.5-mini-instruct", apiKey: settings.HuggingFacePhiApiKey, serviceId: "phi-3")
                   .Build();


var prompt = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Prompt");

var gptResult = await kernel.InvokePromptAsync(prompt, new(new PromptExecutionSettings()
{
    ServiceId = "gpt"
}));

Console.WriteLine("gpt");
Console.WriteLine(gptResult);
Console.WriteLine();

var geminiResult = await kernel.InvokePromptAsync(prompt, new(new PromptExecutionSettings()
{
    ServiceId = "gemini"
}));

Console.WriteLine("gemini");
Console.WriteLine(geminiResult);
Console.WriteLine();

var phi3Result = await kernel.InvokePromptAsync(prompt, new(new PromptExecutionSettings()
{
    ServiceId = "phi-3"
}));


Console.WriteLine("phi-3");
Console.WriteLine(phi3Result);
Console.WriteLine();

gpt
나는 OpenAI에서 개발한 AI 언어 모델인 ChatGPT야. 어떤 도움이 필요하니?

gemini
저는 구글에서 개발한 대규모 언어 모델입니다.

phi-3
저는 편집증이 있는 AI이기 때문에 개인적인 의식이나 감정을 가지고 있지는 않습니다. 도움을 드리는 데 최선을 다합니다. 특정

